# Yaml_Sci_Config

This is a small library based on ruamel.yaml which adds some functionality to your code and allows you to focus more time on implementation and less on parsing inputs.

## Installation

This package is pip installable. To install it, clone the directory and run `pip install PATH` where PATH is the path to the directory. The package is installed as yaml_sci_config.


## Using the Interface Classes

This library has several classes that help you add functionality to code. Each interface class has a tag, which is its class name preceded by an exclamation point. As an example, the FunctionHandle class has tag `!FunctionHandle`. This tag comes before its use in the yaml file.

The following example gives a sneak peek as to what our library can do. We make yaml load a class defined in our interface.



In [1]:
from yaml_sci_config.yaml_interface import yaml_load
from yaml_sci_config.interface_classes import FunctionHandle


yaml_str=\
'''
a: !FunctionHandle
    module_name: numpy
    function_name: sum

'''
config_load = yaml_load(yaml_str)
print(config_load['a'])



FunctionHandle(module_name='numpy', function_name='sum', _fn_hand=<function sum at 0x0000023D9039D1B0>)



### Example Load and dump to yaml: Function Handle

The previous object we created was called a function handle. Let's see what exactly a function handle does.
It should behave in most cases exactly like the function at hand. If you need everything that a function handle needs, though, like __name__ attributes, then call its `to_function_handle` method. (the last line fails because it is not a function handle.


In [2]:
import numpy as np

loaded_numpy_sum = config_load['a']
numpy_sum = np.sum
x = np.array([1,2,3,4,5])
print(f'loaded sum: {loaded_numpy_sum(x)}')
print(f'numpy sum: {np.sum(x)}')

loaded_numpy_sum_handle = loaded_numpy_sum.to_function_handle()

print(f'loaded_to_fn_handle name: {loaded_numpy_sum_handle.__name__}')
print(f'numpy sum name:{np.sum.__name__}')
try:
    print(f'loaded sum name: {loaded_numpy_sum.__name__}')
except AttributeError:
    print(f'whoops. couldn''t do that.')

loaded sum: 15
numpy sum: 15
loaded_to_fn_handle name: sum
numpy sum name:sum
whoops. couldnt do that.


Let's send our custom python handle to print to yaml. We dump to a string with yaml_dumps because it is convenient here. Notice our nested config also prints just fine.

In [3]:
from yaml_sci_config.yaml_interface import yaml_dumps

print(yaml_dumps(config_load['a']))
print (' ')
longer_info_to_dump = {'2':43,'nested_field':{'a':loaded_numpy_sum,'b':'other_string'}}
print(yaml_dumps(longer_info_to_dump))

!FunctionHandle
module_name: numpy
function_name: sum

 
'2': 43
nested_field:
  a: !FunctionHandle
    module_name: numpy
    function_name: sum
  b: other_string



## Interface Builtin classes

### Numpy Array Support

We have a fine selection of numpy array wrappers, and some parsing. Examples of generating them in python and through YAML strings. Most


In [4]:
from yaml_sci_config.interface_classes import LogspaceParams,LinspaceParams
numpy_examples_str= \
'''\
linspace_arr: !LinspaceParams
    lin_start: 1
    lin_stop: 2
    n_linspace: 11
logspace_arr: !LogspaceParams
    log_start: 1 # 10^1
    log_stop: 3  # 10^3
    n_logspace: 7
'''
np_examples = yaml_load(numpy_examples_str)

linspace_arr:LinspaceParams = np_examples['linspace_arr'].to_np_array()
logspace_arr:LogspaceParams = np_examples['logspace_arr'].to_np_array()
print(linspace_arr)
print(logspace_arr)

[1.  1.1 1.2 1.3 1.4 1.5 1.6 1.7 1.8 1.9 2. ]
[  10.           21.5443469    46.41588834  100.          215.443469
  464.15888336 1000.        ]


We also include other examples of parsing.

In [5]:




other_examples_str = \
'''\
list_elem: [2,3, 36]
np_arr_explicit: np.array([2,3,26])
np_complex_arr: np.array([5e2-4.5i,2])
complex_val: 4.2+3i # support for scientific notation, i instead of j
yaml_list:
 - 1
 - !FunctionHandle {"module_name":"numpy", "function_name":"mod"}
'''
other_examples = yaml_load(other_examples_str)

print(other_examples)


{'list_elem': [2, 3, 36], 'np_arr_explicit': array([ 2,  3, 26]), 'np_complex_arr': array([500.-4.5j,   2.+0.j ]), 'complex_val': (4.2+3j), 'yaml_list': [1, FunctionHandle(module_name='numpy', function_name='mod', _fn_hand=<ufunc 'remainder'>)]}




### Function, Partial Function, ClassObject and PartialClassObject

You can import functions or class objects too!




In [6]:
yaml_str='''\
a: !FunctionHandle
    module_name: numpy
    function_name: sum
b: !PartialFunctionHandle
    module_name: numpy
    function_name: sum
    kwargs: {axis: -1}
c: !PartialFunctionHandle # showing different yaml syntax for dictionaries
    module_name: numpy
    function_name: sum
    kwargs:
        axis: 0
'''
import numpy as np
loaded = yaml_load(yaml_str)

np_sum = loaded['a']
np_sum_axis_neg1 = loaded['b']
np_sum_axis_0 = loaded['c']

test_arr = np.array([[1,2,3],[4,5,6]])

all_sum =np_sum(test_arr)
sum_axis_0 = np_sum_axis_0(test_arr)
sum_axis_neg1 = np_sum_axis_neg1(test_arr)
print(f'all sums: {all_sum}, first axis: {sum_axis_0}, second axis: {sum_axis_neg1}')


all sums: 21, first axis: [5 7 9], second axis: [ 6 15]


In [7]:
yaml_str = '''\
d: !ClassObject
    module_name: numpy
    class_name: array
e: !PartialClass
    module_name: numpy
    module_name: numpy
'''
loaded = yaml_load(yaml_str)

AttributeError: 'ClassObject' object has no attribute 'function_name'

## Making Custom Dataclasses

To make a custom dataclass (see python dataclasses documentation), which you can import, is very straightforward.

In [12]:
from yaml_sci_config.yaml_interface import yaml_dataclass
from numpy.typing import ArrayLike

@yaml_dataclass
class CustomDataclass:
    field1: str
    field2: list

@yaml_dataclass
class Circle:
    center: ArrayLike
    r: float
    def __post_init__(self):
        self.center = np.asarray(self.center)
        assert(self.center.shape == (2,))

@yaml_dataclass
class Eyes:
    left_eye: Circle
    right_eye: Circle
    def __post_init__(self):

        assert (np.sum((self.left_eye.center- self.right_eye.center)**2 >=
                       self.left_eye.r + self.right_eye.r))
custom_types_yaml=\
'''
cust_class: !CustomDataclass # make sure the space between the exclamation point and yaml tag is there
    field1: This
    field2: [2,4,5]

circle: !Circle
    center: np.array([2.3,4.1]) # convenience method for numpy arrays. Still should be sanitizied in post_init.
    r: 2.4

eyes: !Eyes
    left_eye: !Circle
        center: np.array([-1.0,0])
        r: .75
    right_eye: !Circle
        center: np.array([1,0.0])
        r: .75
'''

loaded = yaml_load(custom_types_yaml)

print(loaded['cust_class'])
print(loaded['circle'])
print(loaded['eyes'])



CustomDataclass(field1='This', field2=[2, 4, 5])
Circle(center=array([2.3, 4.1]), r=2.4)
Eyes(left_eye=Circle(center=array([-1.,  0.]), r=0.75), right_eye=Circle(center=array([1., 0.]), r=0.75))


# Running a script, generating config files.

Finally,

#